# Sandbox

Experimenting with implementations of a functional API for inverse problems.

## Thoughts

* Hidding things from the user inside classes makes the library more accessible but the code more complex and obscure.
* Maybe go back to a more explicit API where the user needs to know that an optimization method is being used and which one. 
* The bad thing about that is that the code itself doesn't have all the instruction for executing an inversion (just the Jacobian and predicted data probably).
* Can get around this by investing in examples in the documentation.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import dask as da

## Optimization

Functions that optimize a goal function. 

Gradient methods need to be able to calculate the function and its gradient and Hessian matrices. 

Should be separate functions or single function that takes an argument specifying what it wants? Argument in favor of single function is that gradient and Hessian ofter share expensive variables (like Jacobian).

Include stopping criterion or just run for given number of iterations? Usually iterations are expensive so maybe running one at a time and looking at intermediate results is a good thing?

In [5]:
def levmarq(function, initial, iterations, step=2, step_increase=2,
            maxsteps=10, precondition=True):
    p = initial
    lamb = step
    previous_value = functions(p)
    for iteration in range(iterations):
        grad, hess = function(p, calculate=['gradient', 'hessian'])
        deltap = np.linalg.solve(hess, -grad)
        p = p + deltap
        value = function(p)
        if value > previous_value:
            print("Function increased")
            break
        previous_value = value
    return p
        